In [72]:
import pandas as pd
import xarray as xr
import numpy as np
import metacsv
import csv
import StringIO
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import copy
from glob import glob
from scipy.stats import multivariate_normal

1. Users load a config file with three parameters
        1. mode: single, mc, or median. Single seems like it is totally dry-run just to get basic feedback
            A. These refer to number of realizations of full run to do
        2. Specification: Mortality, Labor, Ag, whatever impact
        3. Output dir, where to write this stuff
        
2. This config file is read by generate.generate and calls the specific functions in that file. 

3. Each of the three options call functions that yield the inputs that are then called for the specific impact function that is being specified. Each of these impact sectors is uniquely specified in another file but the interface to the impact functions are all the same.

4. Each discrete calculation regardless of it is single or mc or median need the following inputs
        pvals: the pvalue to sample at 
        clim_scenario: RCP #file identification
        clim_model: CMIP5 model: file identification
        weatherbundle: weather values of tas, pr, wind, etc: file identification
        econ_scenario: high or low# config
        econ_model: SSSP# config
        economicmodel: Wraps a class around the data that comes from econ_scenario and econ_model with access to basic stats about the economic model. Not sure how to get this out but maybe it is simple do_econ_thing decorator
        
        

Behind this is the whole set up of the entire flow of data that unleashes the whole calculation. When get_bundle_iterator is called it goes and queues up all the necessary values for the computation and the generators emit the information one by one. So generate.generate is more infrastructure and input file config. And it sets that config up based on the type of calculation you are runnning. 

Questions
What I don't totally understand is binpush_callback, valpush_callback, binresult_callback, valresult_callback, 

In [ ]:
If the goals of the gcp is to estimate/forecast future joint probability distributions, then the we need to represent those 
distributions as data structures. We need to represent a joint probability distribution for every location for every year
in our dataset. Each of those probability distributions is the result of a number of uncertain/probabalistic inputs.  

In [ ]:
pvals, clim_scenario, clim_model, weatherbundle, econ_scenario, econ_model, economicmodel

In [3]:
outputs_histclim = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_best_14feb-histclim.nc4')

In [244]:
output_labor = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_best_14feb.nc4')
output_labor_c = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_best_14feb_comatose.nc4')
output_labor_d = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_best_14feb_dumb.nc4')

Frozen(OrderedDict([(u'year', <xarray.IndexVariable u'year' (year: 120)>
array([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028,
       2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040,
       2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052,
       2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064,
       2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076,
       2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088,
       2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100], dtype=int32)
Attributes:
    long_title: Impact year (Gregorian calendar)
    units: Years
    source: From the weather file.), (u'regio

In [28]:
metacsv.read_header('/Users/rhodiumgroup/data/gcp_stuff/gdppc-merged.csv')

(<Empty Attributes>, <Empty Coordinates>, <Empty Variables>)

In [71]:
#Use some decorators or closures to deal with inputs that are custom for each run
#

def do_climate_thing(path, climate_arguments):
    '''
    Load climate data year by year, we need tasmax by IR. We have this data. 
    Per run, do this 100 times, read a new dataset in, and take the dot-product

    Parameters
    ----------
    path: str
        path to a netCDF file
        
    climate_arguments: str
        climate variable evaluated 
        
    Returns
    -------
    DataArray

    '''  
    ds = xr.open_dataset(path)
    return ds[climate_arguments]
    
    
    
def do_covariate_thing(*paths):
    '''
    Read in covariate data
    Align datasets along region and time dimension
    Perform any necessary transformations
    For socio-economic data we need to reevaluate every few years 
    
    Parameters
    ----------
    Paths to covariate data
    
    Returns
    -------
    MxN Matrix where M is the number of IRs and N is the number of covariates
    
    '''
    
    #read file
    #take baselines
    #update parameters
    
        
def get_local_surface_estimate_at_given_p_value(IR_annual_covariates, gammas, pval):
    '''
    Element-wise multiplication of gammas and IR-level covariates. 
    Equivalent to the local response functions

    
    Parameters
    ----------
    IR_covariates: MxN matrix where M is the num IRs and N is the covariates
    
    gammas: Mx1 column vector of point estimates of global impact function specification parameters
    
    Returns
    -------
    MxN Matrix where M is IRs and N is local estimate of function specification params

    '''
    
    #Load IR matrix
    #load gammas
    #do math
    #return matrix maybe write to disk or do some diagnostic
    
    
    
    
def do_math_thing(climate_object, IR_parameters,function_specification):
    '''
    Do math according to function specification
    
    Parameters
    ----------
    Climate_object: Xarray DataSet or Xarray DataArray
    
    IR_parameters: MxN matrix of regions by parameters 
    
    function_specification: Arbitrary function that takes Climate_object and IR_params as args
    
    Returns
    -------
    Xarray object: result of computation as a MxN Matrix where M is the number of IRs and N is number responses
                   represents one year of impacts for each IR
    
    '''
    
    return function_specification(climate_object, IR_parameters)

def do_collect_results_thing(annual_responses_by_IR_object, temp_path_on_disk, output_dir=None):
    '''
    Collects annual results into larger dataset
    
    Parameters
    ----------
    annual_responses_by_IR_object: Xarray DataSet
        represents annual impacts
        
    temp_path_on_disk
    -----------------
        
    output_dir: str
        Location to write output
    
    Returns
    -------
    N-dimensional data set or None
    
    '''
    #append annual_responses_by_IR_object to file on disk
    ds = xr.open_dataset(path_on_disk)
    #ds.update(annual_response_by_IR_object) 
    
    if output_dir is not None:
        ds.write_to_netcdf(output_dir)
    return ds
    

def to_datafs(api_object, archive_name, ds= None,path=None, cache=False)
    '''
    Creates a record in datafs and uploads file to osdc. If cache is True will save to cache
    
    Parameters
    ----------
    api_object: initialized api with users parameters from local datafs config file
    
    archive_name: str
        name of archive to create in datafs
        
    ds: Xarray Object
        dataset to be created and uploaded to datafs 
    
    path: str
        if None then archive is created from memory
    
    cache=bool
        if True, save file to cache
    
    '''

def do_pipeline(init_dict):
    '''
    Parameters
    ----------
    init_dict: dict
        Specifies the parameters of the subroutines in do_pipeline  
        
    Returns
    -------
    None
    Creates archive(s) in datafs that correspond to parameters in init_dict
    
    
    1.get coefficients from the csvv draw via np.multinomial.rvs(gammas, covariates)
    2.multiply coefficients by values in socioeconomics for each region 
    3.when we multiply the IR-level socio values and the gammas we get an IR-level curve
    4.take the dot product of the climate variable at that order of magnitude 
    '''
    
    

In [133]:
degree_days = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/Degreedays_aggregated_rcp45_r1i1p1_CCSM4.nc')

coldd = degree_days.coldd_agg
hotdd = degree_days.hotdd_agg

In [202]:
coldd

<xarray.DataArray 'coldd_agg' (time: 119, SHAPENUM: 24378)>
[2900982 values with dtype=float32]
Coordinates:
  * SHAPENUM  (SHAPENUM) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Dimensions without coordinates: time
Attributes:
    long_title: aggregation ofCold degree days that tasmax is less than 10 degree celsuisin impact regions
    units: degree days
    source: Regional aggregated coldd from /global/scratch/jiacany/nasa_bcsd/Labor/degree_days/tasmax/rcp45/CCSM4/tasmax_exceedance_degree_days_rcp45_r1i1p1_CCSM4_1.nc

In [246]:
#tasmax = glob('/Users/rhodiumgroup/data/gcp_stuff/tasmax_day_aggregated_rcp45_r1i1p1_CCSM4_2006.nc')

In [65]:
pop_baseline = pd.read_csv('/Users/rhodiumgroup/data/gcp_stuff/popop_baseline.csv')

In [90]:
filename = '/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_3factor_BEST_14feb.csvv'
csvv = read(filename)

In [251]:
tasmax = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/tasmax_day_aggregated_rcp45_r1i1p1_CCSM4_2006.nc')
tasmax

<xarray.Dataset>
Dimensions:   (SHAPENUM: 24378, time: 365)
Coordinates:
  * SHAPENUM  (SHAPENUM) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * time      (time) float32 2.006e+06 2.006e+06 2.006e+06 2.006e+06 ...
Data variables:
    tasmax    (time, SHAPENUM) float32 -10.7914 -10.6006 0.773431 -12.0515 ...
Attributes:
    version: BCSD.2016-02-21
    description: GCP regional agrregated data
    dependencies: Agglomerated-Many.2016-02-17NASA-GDDP
    contact: Jiacan Yuan (jiacan.yuan@gmail.com), Rutgers University

In [109]:
collapse_bang(csvv, 10)

In [110]:
subs = subset(csvv, prednames)

In [192]:
hdd = hotdd[0].to_dataframe()
hdd.index -= 1
hdd.head()

,hotdd_agg
SHAPENUM,
0,2.0
1,5.0
2,0.0
3,0.0
4,0.0


In [194]:
gdppc = pd.DataFrame(np.random.randint(1000,23000, 24378))
gdppc['hierid'] = popop['hierid']
gdppc.columns = ['gdppc', 'hierid']
gdppc.head()

,gdppc,hierid
0,20631,CAN.1.2.28
1,8329,CAN.1.17.403
2,2101,CAN.2.34.951
3,17090,CAN.11.259.4274
4,13741,CAN.11.269.4448


In [222]:
popop = pop_baseline[['hierid', 'popop']]
popop.head()
merged = pd.merge(popop, gdppc, on='hierid')

<xarray.DataArray 'coldd_agg' (SHAPENUM: 24378)>
array([ 2312.,  2076.,  1161., ...,     0.,     0.,     0.], dtype=float32)
Coordinates:
  * SHAPENUM  (SHAPENUM) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Attributes:
    long_title: aggregation ofCold degree days that tasmax is less than 10 degree celsuisin impact regions
    units: degree days
    source: Regional aggregated coldd from /global/scratch/jiacany/nasa_bcsd/Labor/degree_days/tasmax/rcp45/CCSM4/tasmax_exceedance_degree_days_rcp45_r1i1p1_CCSM4_1.nc

In [223]:
merged['hotdd'] = hotdd[0]
merged['coldd'] = coldd[0]

In [238]:
#for every year/region calculation you'd have to create something like this to get local
#Should we create a data structure that is just IR-levelbetas
merged_new  = merged.ix[:,1:5]
merged_new.values[0]

array([  6.06338691e+02,   2.06310000e+04,   2.00000000e+00,
         2.31200000e+03])

In [239]:
IR_curves= []
for k,v in predgammas.items():
    IR_curve = {}
    for i in merged_new.values[0]:
        IR_curve[i] = {}
        IR_curve[i][k] = merged_new.values*v
        

In [240]:
IR_curve

{2.0: {'tasmax2': array([[ -7.32877888e+02,  -3.49285426e+03,   2.05635875e+00,
            3.08205891e+02],
         [ -1.24345787e+02,  -1.41011018e+03,   5.14089687e+00,
            2.76745428e+02],
         [ -1.30527697e+01,  -3.55701944e+02,   0.00000000e+00,
            1.54769481e+02],
         ..., 
         [ -1.68765919e+03,  -7.99609843e+02,   4.25955248e+02,
            0.00000000e+00],
         [ -6.65730516e+02,  -9.72466427e+02,   4.55237494e+02,
            0.00000000e+00],
         [ -7.44772535e+02,  -9.68233896e+02,   5.20143451e+02,
            0.00000000e+00]])},
 606.33869120000008: {'tasmax2': array([[ -7.32877888e+02,  -3.49285426e+03,   2.05635875e+00,
            3.08205891e+02],
         [ -1.24345787e+02,  -1.41011018e+03,   5.14089687e+00,
            2.76745428e+02],
         [ -1.30527697e+01,  -3.55701944e+02,   0.00000000e+00,
            1.54769481e+02],
         ..., 
         [ -1.68765919e+03,  -7.99609843e+02,   4.25955248e+02,
            0.00000

In [231]:
predgammas.keys()


['tasmax', 'tasmax4', 'tasmax3', 'tasmax2']

In [115]:
constant

{'tasmax': -33.912498866251397,
 'tasmax2': 5.2971225130931705,
 'tasmax3': -1.7344001597803684,
 'tasmax4': -0.0058674426793554708}

In [116]:
predcovars

{'tasmax': ['loggdppc', 'logpopop', 'hotdd_agg', 'coldd_agg'],
 'tasmax2': ['loggdppc', 'logpopop', 'hotdd_agg', 'coldd_agg'],
 'tasmax3': ['loggdppc', 'logpopop', 'hotdd_agg', 'coldd_agg'],
 'tasmax4': ['loggdppc', 'logpopop', 'hotdd_agg', 'coldd_agg']}

In [81]:
local_curve = 

In [51]:
#Curve_logic

# The conceptual goal of interpolation-and-adaptation
# process in the GCP is to estimate response curves at some unknown
# location and future time.  So, those response curves are embodied as
# instances of the Curve class.  Whatever that process is to generate
# those response curves is embodied in a CurveGenerator class.


#To get the inputs to this curve, we read in the csvv file take the gammas and then the diagonal of the covariance matrix


def read(filename):
    with open(filename, 'rU') as fp:
        attrs, coords, variables = metacsv.read_header(fp, parse_vars=True)
        data = {'attrs': attrs, 'variables': variables, 'coords': coords}

        if 'csvv-version' in attrs:
            if attrs['csvv-version'] == 'girdin-2017-01-10':
                return read_girdin(data, fp)

     
def read_girdin(data, fp):
    reader = csv.reader(fp)
    variable_reading = None

    for row in reader:
        if len(row) == 0 or (len(row) == 1 and len(row[0].strip()) == 0):
            continue

        if row[0] in ['observations', 'prednames', 'covarnames', 'gamma', 'gammavcv', 'residvcv']:
            data[row[0]] = []
            variable_reading = row[0]
        else:
            if variable_reading is None:
                print "No variable queued."
                print row
            assert variable_reading is not None
            data[variable_reading].append(map(lambda x: x.strip(), row))

    data['observations'] = float(data['observations'][0][0])
    data['prednames'] = data['prednames'][0]
    data['covarnames'] = data['covarnames'][0]
    data['gamma'] = np.array(map(float, data['gamma'][0]))
    data['gammavcv'] = np.array(map(lambda row: map(float, row), data['gammavcv']))
    data['residvcv'] = np.array(map(lambda row: map(float, row), data['residvcv']))

    return data

In [52]:
def collapse_bang(data, seed):
    if seed == None:
        data['gammavcv'] = None
    else:
        np.random.seed(seed)
        data['gamma'] = multivariate_normal.rvs(data['gamma'], data['gammavcv'])
        data['gammavcv'] = None

In [53]:
def subset(csvv, prednames):
    toinclude = map(lambda predname: predname in prednames, csvv['prednames'])
    toinclude = np.where(toinclude)[0]

    subcsvv = copy.copy(csvv)
    subcsvv['prednames'] = [csvv['prednames'][ii] for ii in toinclude]
    subcsvv['covarnames'] = [csvv['covarnames'][ii] for ii in toinclude]
    subcsvv['gamma'] = csvv['gamma'][toinclude]
    if 'gammavcv' in csvv and csvv['gammavcv'] is not None:
        subcsvv['gammavcv'] = csvv['gammavcv'][toinclude, toinclude]

    return subcsvv

In [54]:
prednames = ['tasmax', 'tasmax2', 'tasmax3', 'tasmax4']

In [10]:
ind = 'C'
dep = 'minutes worked by individual', 
prefix = 'tasmax' 
order = 4

In [193]:
#subs['gamma'] = gamma
#subs

In [113]:
# Preprocessing
constant = {} # {predname: constant}
predcovars = {} # {predname: [covarname]}
predgammas = {} # {predname: np.array}
for predname in set(prednames):
    constant[predname] = 0
    predcovars[predname] = []
    predgammas[predname] = []
    
    indices = [ii for ii, xx in enumerate(subs['prednames']) if xx == predname]
    for index in indices:
        if subs['covarnames'][index] == '1':
                    constant[predname] += subs['gamma'][index]
        else:
            predcovars[predname].append(subs['covarnames'][index])
            predgammas[predname].append(subs['gamma'][index])
    predgammas[predname] = np.array(predgammas[predname])

In [51]:
def get_coefficients(covariates, debug=False):
    coefficients = {} # {predname: sum}
    for predname in set(prednames):
        if len(predgammas[predname]) == 0:
            coefficients[predname] = np.nan
        else:
            try:
                coefficients[predname] = constant[predname] + np.sum(predgammas[predname] * np.array([covariates[covar] for covar in predcovars[predname]]))
            except Exception as e:
                print "Available covariates:"
                print covariates
                raise e

    return coefficients

In [24]:
#subs

In [ ]:
#Read from gammas and associated covariance matrix. 
# X Mathematical operations: 
    #one: When calling get_curve for the FarmerCurveGenerator you are actually calling 
    # np.polyval

    #one:line 46 of curvegen.CSVVCurvegen: 
            #1. Defines the calculation for each coefficient according to the type of distribution that will be computed
            #2. Reads the csvv and sets up the datastructure
            #3. I don't see where the numerica values from the csvv are computed until
        #Generates a coefficient for each predictor so we'll have a dict
        #Is this a set of predictor: coefficient key-value pairs for each region, for each year
        #{'tasmax': someval, 'tasmax2': someval, 'tasmax3': some_val, 'tasmax4': some_val}
        #Each coefficient (tasmax, tasmax2, tasmax3 etc) is the sum of some constant and the 
        #product of that tasmax*loggdp + tasmax*hotdd + tasmax*coldd
    #two: Then call this Farmer curve generator which basically takes the 
        #dictionary of coefficients and the computation that generates them adds them to a list
        # For each region there is this list of dictionaries which represent the a specified calculation 
        # coefficient on the predictor climate variable.
        #Then that is handed to np.polyval
        #Can you point me to a get_cruve method that is implemented? I'd like to understand the 
        #the operation that specifies the curve. 
        #Region Curves: List of curves defined by region specific output of np.polyval
#Generate a curve for Farmer: 
#ge

In [ ]:
When we get to curve generation there are two module-level curve generation calls
1. PolynomialCurveGeneratro(or MLECubicSplineGenerator), this takes in the modified data structure that came from the csvv
subcsvv now is 

In [241]:
finals = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_3factor_best_14feb.nc4')

<xarray.DataArray 'response' (region: 24378)>
array([  44.209047,   76.841777,   40.760857, ...,  237.901736,  292.757717,
        316.413947])
Coordinates:
    year     int32 1981
Dimensions without coordinates: region
Attributes:
    long_title: Direct marginal response
    units: minutes worked by individual
    source: The average result across a year of daily temperatures applied to a polynomial.